# 网络编程

现在基本上所有的程序都是网络程序了，很少有单机版本的程序了。

计算机网络就是把各个计算机连接到一起，让网络中的计算机可以互相通信。网络编程就是如何在程序中实现两台计算机的通信。

由于计算机上实际运行的实体是进程，所以**网络通信实现上就是两个进程之间通过网络在进行通信**。


## 1. TCP/IP协议簇

Internet网络之前就已经存在计算机网络了，初期的计算机网络更多是在一些大型的学校、企业机构用于内部的多计算机之间的互联与网络通信。它们各自都有自己的协议。

TCP/IP协议簇成为了默认的通用协议标准。任何私有网络，只要支持这个协议，就可以联入网联网。

## 2. IP地址

在一个大的网络中，通信的两台计算机要能够互相识别出对方，所以每台计算机就必须有个标识，这个标识就是IP地址。但如果一台计算机同时接入到了两个或更多的网络，比如路由器，它就会有两个或多个IP地址。所以,**IP地址对应的实际是计算机的网络接口，通常是网卡**。

## 3. TCP协议

TCP协议则是建立在IP协议之上的。TCP协议负责在两台计算机之间建立可靠连接，保证数据包按顺序到达。TCP协议会通过握手建立连接，然后，对每个IP包编号，确保对方按顺序收到，如果包丢掉了，就自动重发。

一个TCP报文除了包含要传输的数据外，还包含源IP地址和目标IP地址，源端口和目标端口。

一个进程也可能同时与多个计算机建立链接，因此它会申请很多端口。

### 3.1 TCP服务器端

[Code: TCP Server](../src/tcp_server.py)

#### 3.1.1 创建TCP的socket

In [1]:
import socket

# AF_INET指定的是IPv4的协议，如果是IPv6的协议，就指定为AF_INET6
# SOCK_STREAM指定的是使用面向流的TCP协议
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#### 3.1.2 服务器端绑定IP地址与端口

服务器可能有多块网卡，可以绑定到某一块网卡的IP地址上，也可以用`0.0.0.0`绑定到所有的网络地址，还可以用`127.0.0.1`绑定到本机地址。`127.0.0.1`是一个特殊的IP地址，表示本机地址，如果绑定到这个地址，客户端必须同时在本机运行才能连接，也就是说，外部的计算机无法连接进来。

In [2]:
 s.bind(('127.0.0.1', 9999))

#### 3.1.3 开始监听

传入的参数指定等待连接的最大数量。这个参数代表什么意思呢。

每当有一个客户端`connect`了，`listen`的队列中就加入一个连接，每当服务器端`accept`了，就从`listen`的队列中取出一个连接，转成一个专门用来传输数据的`socket`（`accept`函数的返回值）

In [3]:
s.listen(5)
print('waiting for connection...')

waiting for connection...


#### 3.1.4 处理连接请求

接下来服务器将通过一个永久的循环来接受来自客户端的连接，`accept()`会等待并返回一个客户端连接。

In [ ]:
def server_echo(sock, addr):
    print('Accept new connection from %s' % addr)
    sock.send(b'Welcome!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        sock.send(data)
    sock.close()
    print('connection from %s closed.' % addr)
while True:
    conn, addr = s.accept()
    t = threading.Thread(target=tcplink, args=(conn, addr))
    t.start()

### 3.2 TCP-客户端

[Code: TCP Client](../src/tcp_client.py)

#### 3.2.1 与服务器端建议连接

In [28]:
s.connect(('www.baidu.com', 80))

#### 3.2.2 向服务器发送数据

In [29]:
s.send(b'GET / HTTP/1.1\r\nHost: www.baidu.com\r\nConnection: close\r\n\r\n')

58

#### 3.2.3 从服务器接收数据

接收数据时，调用recv(max)方法，一次最多接收指定的字节数，因此，在一个while循环中反复接收，直到recv()返回空数据，表示接收完毕，退出循环。

In [20]:
buffer =[]
while True:
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break
data = b''.join(buffer)
header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
#print(html.decode('utf-8'))

HTTP/1.1 200 OK
Accept-Ranges: bytes
Cache-Control: no-cache
Content-Length: 14615
Content-Type: text/html
Date: Fri, 25 Oct 2019 01:46:03 GMT
Etag: "5d8b1fec-3917"
Last-Modified: Wed, 25 Sep 2019 08:06:04 GMT
P3p: CP=" OTI DSP COR IVA OUR IND COM "
Pragma: no-cache
Server: BWS/1.1
Set-Cookie: BAIDUID=568F8783435B61209AEBCD7ABEF727C4:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com
Set-Cookie: BIDUPSID=568F8783435B61209AEBCD7ABEF727C4; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com
Set-Cookie: PSTM=1571967963; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com
Vary: Accept-Encoding
X-Ua-Compatible: IE=Edge,chrome=1
Connection: close


#### 3.2.4 关闭socket

In [ ]:
s.close()

## 4. UDP协议

TCP是建立可靠连接，并且通信双方都可以以流的形式发送数据。相对TCP，UDP则是面向无连接的协议。

使用UDP协议时，不需要建立连接，只需要知道对方的IP地址和端口号，就可以直接发数据包。但是，能不能到达就不知道了。

### 4.1 UDP服务器端

[Code: UDP Server](../src/udp_server.py)

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.bind(('127.0.0.1', 9998))
while True:
    data, addr = s.recvfrom(1024)
    print('Received from %s:%s.' % addr)
    s.sendto(b'Hello, %s' % data, addr)

### 4.2 UDP客户端

[Code: UDP Client](../src/udp_client.py)

UDP的发送使用了`sendto`函数，实际上也可以使用`send`函数，前提是对于`socket`要调用`connect`函数，让其绑定要发送数据的服务端。

`recv`函数也可以换成`recvfrom`函数。

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
for data in [b'Michael', b'Tracy', b'Sarah']:
    s.sendto(data, ('127.0.0.1', 9998))
    print(s.recv(1024).decode('utf-8'))
s.close()